In [1]:
# Importing libraries

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import datetime

from scipy import stats
from scipy.stats import skew
from scipy.stats import norm
from scipy.stats.stats import pearsonr

from sklearn.ensemble import RandomForestRegressor

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Option to display all the dataframe columns
pd.options.display.max_columns = None

# Importing dataset
TRAIN_PATH = "dataset/train_original.csv"
TEST_PATH = "dataset/test.csv"

df_train = pd.read_csv(TRAIN_PATH)
df_test = pd.read_csv(TEST_PATH)

train_instances = df_train.shape[0]
test_instances = df_test.shape[0]

In [2]:
# Concatenate train and test to perform preprocessing on both

df = pd.concat([df_train, df_test], sort=False).reset_index()

# Transform dates from '%d/%m/%Y' to datetime objects.

def transform_date(x):
    date = datetime.datetime.strptime(x, '%d/%m/%Y')
    return date

df['Date'] = df['Date'].map(transform_date)

# Transform categorical features

df["StoreType"] = df["StoreType"].map({'Standard Market': 0, 'Super Market': 1, 'Hyper Market': 2, 'Shopping Center': 3})
df = pd.get_dummies(df, columns=['AssortmentType'])

for e in ['Rain', 'Snow', 'Fog', 'Hail', 'Thunderstorm']:
    df[e] = df.loc[:, 'Events'].astype('str').apply(lambda x: int(e in x))
    
df = df.drop(labels=['NumberOfCustomers', 'Region', 'AssortmentType_General', 'Events'], axis=1)
    
# Add date-related features

def is_saturday(day):
    return int(day.weekday() == 5)

def is_sunday(day):
    return int(day.weekday() == 6)

df['IsSaturday'] = df['Date'].map(is_saturday)
df['IsSunday'] = df['Date'].map(is_sunday)    

def ordinal_date(x):
    d = x['Date']
    date = datetime.date(d.year, d.month, d.day)
    return datetime.date.toordinal(date)

df['OrdinalDate'] = df.loc[:, ['Date']].apply(ordinal_date, axis=1)

base_date = min(df['OrdinalDate'])

df['OrdinalDate'] = df['OrdinalDate'].apply(lambda x: x - base_date)

# Divide dataset in chunks based on StoreID

stores = []
for store in df['StoreID'].unique():
    stores.append( df.loc[(df['StoreID'] == store)] )
    
def was_open_yesterday(x):
    date = x['OrdinalDate']
    store_index = x['StoreID'] - 1000
    if date < 30:
        return 1
    that_date = stores[store_index].loc[(stores[store_index]['OrdinalDate'] == date - 1)]
    if len(that_date) < 1:
        return 0
    else:
        return that_date['IsOpen'].item()
    
df['WasOpenYesterday'] = df.loc[:, ['OrdinalDate', 'StoreID']].apply(was_open_yesterday, axis=1)

def is_open_tomorrow(x):
    date = x['OrdinalDate']
    store_index = x['StoreID'] - 1000
    if date == 729:
        return 1
    that_date = stores[store_index].loc[(stores[store_index]['OrdinalDate'] == date + 1)]
    if len(that_date) < 1:
        return 0
    else:
        return that_date['IsOpen'].item()

df['IsOpenTomorrow'] = df.loc[:, ['OrdinalDate', 'StoreID']].apply(is_open_tomorrow, axis=1)

# Drop rows where the store is closed (IsOpen = 0)

df = df.drop(df[df.IsOpen == 0].index)
df = df.drop(labels=['IsOpen'], axis=1)

# Add feature for the mean sales for that month

df['Month'] = df.Date.map(lambda d: d.strftime('%m'))
mean_month_sales = df.groupby('Month').mean().NumberOfSales.to_dict()
df['MeanMonthSales'] = df.Date.map(lambda d: mean_month_sales[d.strftime('%m')])
df = df.drop(labels=['Month'], axis=1)

# Add feature for the mean sales for that store

mean_store_sales = dict()

for store_id in df.StoreID.unique():
    rows = df[df.StoreID == store_id]
    mean_store_sales[store_id] = rows.NumberOfSales.mean()
    
df['MeanStoreSales'] = df.StoreID.map(mean_store_sales)

# Add feature for the std of sales for that store

std_store_sales = dict()

for store_id in df.StoreID.unique():
    rows = df[df.StoreID == store_id]
    std_store_sales[store_id] = rows.NumberOfSales.std()
    
df['StdStoreSales'] = df.StoreID.map(std_store_sales)

# Impute missing values with RandomForestRegressor

def impute_missing(df, feature):
    # Rearrange columns and pick features subset for prediction
    cols = df.columns.tolist()
    cols.insert(0, cols.pop(cols.index(feature)))
    cols.pop(cols.index('Date'))
    cols.pop(cols.index('StoreID'))
    
    na_cols = df.columns[df.isna().any()].tolist()
    na_cols.pop(na_cols.index(feature))
    
    cols = [item for item in cols if item not in na_cols]
    
    feature_df = df[cols]
    
    # Split into sets with known and unknown feature values
    known = feature_df.loc[ (df[feature].notnull()) ]
    unknown = feature_df.loc[ (df[feature].isnull()) ]
    
    y = known.values[:, 0]
    X = known.values[:, 1::]
    
    # Create and fit a model
    rtr = RandomForestRegressor(n_estimators=100, n_jobs=-1)
    rtr.fit(X, y)
    
    # Predict missing values
    predicted = rtr.predict(unknown.values[:, 1::])
    
    # Assign those predictions to the full data set
    df.loc[ (df[feature].isnull()), feature ] = predicted
    
    return df

df = impute_missing(df, 'Min_VisibilitykM')
df = impute_missing(df, 'Mean_VisibilityKm')
df = impute_missing(df, 'Max_VisibilityKm')
df = impute_missing(df, 'CloudCover')
df = impute_missing(df, 'Max_Gust_SpeedKm_h')

In [23]:
df_train         = df.loc[(df['Date'] <  '2018-03-01')]
df_test          = df.loc[(df['Date'] >= '2018-03-01')]
df_train_no_val2 = df_train.loc[(df_train['Date'] <  '2018-01-01')]
df_val2          = df_train.loc[(df_train['Date'] >= '2018-01-01')]

# Outlier elimination

def remove_outliers(df):
    df = df.drop(df[(df.NumberOfSales > df.MeanStoreSales+4.5*df.StdStoreSales) |
                    (df.NumberOfSales < df.MeanStoreSales-3*df.StdStoreSales)].index)
    
    return df

df_train         = remove_outliers(df_train)
df_train_no_val2 = remove_outliers(df_train_no_val2)

msk              = np.random.rand(len(df_train_no_val2)) < 0.9
df_val1_rand     = df_train_no_val2[~msk]
df_train_rand    = df_train_no_val2[msk]

In [24]:
df_train.to_csv("processed/train.csv", index=False)
df_test.to_csv("processed/test.csv", index=False)
df_train_no_val2.to_csv("processed/train_no_val2.csv", index=False)
df_val2.to_csv("processed/val2.csv", index=False)
df_val1_rand.to_csv("processed/val1_rand.csv", index=False)
df_train_rand.to_csv("processed/train_rand.csv", index=False)